In [10]:
import pandas as pd
import numpy as np
import string
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
import pickle
import zipfile

In [11]:
!pip install kaggle
!kaggle datasets download jp797498e/twitter-entity-sentiment-analysis -p dataset

Defaulting to user installation because normal site-packages is not writeable
Dataset URL: https://www.kaggle.com/datasets/jp797498e/twitter-entity-sentiment-analysis
License(s): CC0-1.0
100%|██████████████████████████████████████| 1.99M/1.99M [00:01<00:00, 1.53MB/s]
100%|██████████████████████████████████████| 1.99M/1.99M [00:01<00:00, 1.34MB/s]


In [12]:
with zipfile.ZipFile('./dataset/twitter-entity-sentiment-analysis.zip','r') as f:
    f.extractall("dataset")

In [13]:
data=pd.read_csv("./dataset/twitter_training.csv",header=None)[:10000]
data.rename(columns={0: 'Id', 1: 'Company',2:'Sentiment',3:'Comment'}, inplace=True)
data=data.iloc[:,-2:]
data

,Sentiment,Comment
0,Positive,im getting on borderlands and i will murder yo...
1,Positive,I am coming to the borders and I will kill you...
2,Positive,im getting on borderlands and i will kill you ...
3,Positive,im coming on borderlands and i will murder you...
4,Positive,im getting on borderlands 2 and i will murder ...
...,...,...
9995,Irrelevant,So the smaller studio version of why Xbox Seri...
9996,Positive,Xbox looking BEAUTIFUL
9997,Positive,Xbox looks BEAUTIFUL
9998,Positive,Xbox looking BEAUTIFUL


In [14]:
display(data.isna().sum())
data.dropna(inplace=True)
display(data.isna().sum())

Sentiment      0
Comment      112
dtype: int64

Sentiment    0
Comment      0
dtype: int64

In [15]:
def preprocess(text):
    x=" ".join([st.stem(x) for x in text.split()])
    x=x.translate(str.maketrans('','',string.punctuation))
    return x

In [16]:
target={j:i for i,j in enumerate(set(data["Sentiment"]))}
x=data["Comment"]
y = np.array(data["Sentiment"].map(target))
st=PorterStemmer()
tokenizer=TfidfVectorizer(stop_words='english')
x=x.map(preprocess)
token_mod=tokenizer.fit(x)
tokenized=token_mod.transform(x)

In [17]:
out=pd.DataFrame(tokenized.toarray(), columns=tokenizer.get_feature_names_out())
out["Sentiment"]=y
out.to_csv("preprocessed.csv",index=False)

In [18]:
with open("token_model.pkl","wb") as f:
    pickle.dump(token_mod,f)

In [19]:
target

{'Irrelevant': 0, 'Negative': 1, 'Neutral': 2, 'Positive': 3}